In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import os

import numpy as np

df=pd.read_csv('./data/train.csv')

In [2]:
def limpia_catergoricas(data):

    def limpia_color(df):
        return df.apply(lambda x: abs(ord(x.upper())-ord('Z')))


    def limpia_corte(df):
        dict_cut={'Ideal':5,'Premium':4,  'Very Good':3,'Good':2,  'Fair':1, 'Poor':0}
        return df.map(dict_cut)

    def limpia_claridad(df):
        dict_clarity={'I3':0, 'I2':1, 'I1':2, 'SI2':3, 'SI1':4, 'VS2':5, 
                    'VS1':6, 'VVS2':7, 'VVS1':8, 'IF':9,'F':10}
        return df.map(dict_clarity)

    data.color=limpia_color(data.color)
    data.cut=limpia_corte(data.cut)
    data.clarity=limpia_claridad(data.clarity)
    return data
    

def limpia_dimensiones(data):
    data.columns=['id', 'carat', 'cut', 'color', 'clarity', 'depth_p', 'table_p', 'x', 'y',
       'z', 'price']
    return data[['cut', 'color', 'clarity', 'carat', 'depth_p', 'table_p','price']]

def incluye_ideal_table(data,ideal):
    data['ideal_table']=abs(data.table_p-ideal)
    return data.drop('table_p',axis=1)


In [3]:
df=limpia_catergoricas(df)
df=limpia_dimensiones(df)

df_size = df[df.carat - np.floor(df.carat) == 0]
df_off = df[df.carat - np.floor(df.carat) != 0]
#df=df_size
#df=df_off
df

,cut,color,clarity,carat,depth_p,table_p,price
0,4,19,6,1.01,62.7,58.0,8.754
1,2,19,5,0.60,64.3,57.0,7.492
2,4,18,3,1.60,59.4,59.0,9.010
3,3,16,6,0.30,63.5,58.0,6.227
4,3,20,3,1.00,62.8,57.0,8.314
...,...,...,...,...,...,...,...
40450,5,21,5,0.51,62.5,55.0,7.412
40451,5,18,5,0.80,62.1,54.0,8.016
40452,4,19,3,1.53,58.7,59.0,9.070
40453,5,17,5,1.51,61.6,54.0,9.199


In [4]:
# HyperOpt-Sklearn
from hpsklearn import HyperoptEstimator, sgd

# HyperOpt
from hyperopt import hp

# Sklearn stuff
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np

# Load and split dataset
x, y = load_breast_cancer(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=23)

# Penalizer for SGD
penalty = 'l2'

# Search space of "loss functions"
# Different loss variations are defined, each one
# with an specific probability to be selected
# In this case, "hinge" is prioritized with respect the others
loss = hp.pchoice('loss', [(0.50, 'hinge'), 
                           (0.25, 'log'), 
                           (0.25, 'huber')])

# Search space for alpha value is definied
# with a log-uniform function
alpha = hp.loguniform('alpha', low=np.log(1e-5), high=np.log(1))

# Init Estimator
# The classifier is passed with its respective hyperparameters
model = HyperoptEstimator(classifier=sgd('my_sgd', 
                                         penalty=penalty, 
                                         loss=loss, 
                                         alpha=alpha), 
                          max_evals=10, 
                          trial_timeout=30)

# Training
model.fit(x_train, y_train)

# Results
print(f"Training score: {model.score(x_train, y_train)}")
print(f"Test score: {model.socre(x_test, y_test)}")

# Optimal configuration
print(f"Optimal configuration: {model.best_model()}")

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]job exception: __init__() got an unexpected keyword argument 'n_iter'

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]


TypeError: __init__() got an unexpected keyword argument 'n_iter'